In [1]:
import fitz
import os
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle

In [10]:

def extraer_texto_de_pdfs(carpeta):
    textos = {}
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".pdf"):
            ruta_pdf = os.path.join(carpeta, archivo)
            doc = fitz.open(ruta_pdf)
            texto = ""
            for pagina in doc:
                texto += pagina.get_text()
            textos[archivo] = texto
    return textos

In [11]:
carpeta_pdfs = "ds_pdf"
textos_por_archivo = extraer_texto_de_pdfs(carpeta_pdfs)
for nombre, texto in textos_por_archivo.items():
    print(f"Texto extraído de {nombre}:\n{texto[:100]}...\n")  # Muestra los primeros 100 caracteres

Texto extraído de 1._fundamentos_de_hojas_de_clculo.pdf:
1. Fundamentos de hojas de cálculo
1
1. Fundamentos de hojas de 
cálculo
1.1 Cómo acceder a los dato...

Texto extraído de 1_Resumen_del_captulo_Lectura_y_visualizacin_de_datos.pdf:
Resumen del capítulo: Lectura y visualización de datos
1
Resumen del capítulo: Lectura y 
visualizac...

Texto extraído de 2.Resumen_del_captulo_Teora_de_la_probabilidad.pdf:
Resumen del capítulo: Teoría de la probabilidad
1
Resumen del capítulo: Teoría de 
la probabilidad
E...

Texto extraído de 2._limpieza_de_datos_y_preprocesamiento.pdf:
2. Limpieza de datos y preprocesamiento
1
2. Limpieza de datos y 
preprocesamiento
2.1 Comprensión d...

Texto extraído de 2_Hoja_informativa_Estadstica_descriptiva.pdf:
Hoja informativa: Estadística descriptiva
1
Hoja informativa: Estadística 
descriptiva
Práctica
# Cr...

Texto extraído de 2_Resumen_del_captulo_Estadstica_descriptiva.pdf:
Resumen del capítulo: Estadística descriptiva
1
Resumen del capítulo: 
Esta

In [12]:
with open("todo_el_texto.txt", "w", encoding="utf-8") as f:
    for nombre, texto in textos_por_archivo.items():
        f.write(f"\n### {nombre} ###\n")
        f.write(texto)

In [13]:
def dividir_en_fragmentos(texto, largo=500, solapamiento=100):
    fragmentos = []
    inicio = 0
    while inicio < len(texto):
        fin = inicio + largo
        fragmento = texto[inicio:fin]
        fragmentos.append(fragmento.strip())
        inicio += largo - solapamiento
    return fragmentos

# Aplicación a todos los textos
todos_los_fragmentos = []
for nombre_pdf, texto in textos_por_archivo.items():
    partes = dividir_en_fragmentos(texto)
    for i, fragmento in enumerate(partes):
        todos_los_fragmentos.append({
            "documento": nombre_pdf,
            "id": f"{nombre_pdf}_chunk{i}",
            "texto": fragmento
        })

# Ejemplo de uno
print(todos_los_fragmentos[0])


{'documento': '1._fundamentos_de_hojas_de_clculo.pdf', 'id': '1._fundamentos_de_hojas_de_clculo.pdf_chunk0', 'texto': '1. Fundamentos de hojas de cálculo\n1\n1. Fundamentos de hojas de \ncálculo\n1.1 Cómo acceder a los datos de una hoja de cálculo\nFormatos comunes de archivos de hojas de cálculo\n.xlsx  (Excel): adecuado para análisis de datos complejos con funciones \nde fórmulas, gráficos y distintas opciones de formato.\n.csv  (es decir, "comma-separated values" o valores separados por \ncomas): formato simplificado, solo texto, ideal para la representación \ntabular de datos y una amplia compatibilidad de programas.'}


In [14]:
# Cargar el modelo
modelo = SentenceTransformer('all-MiniLM-L6-v2')

# Extraer los textos
textos = [f["texto"] for f in todos_los_fragmentos]

# Crear embeddings
embeddings = modelo.encode(textos, show_progress_bar=True)

# Crear el índice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Guardar el índice y los metadatos
faiss.write_index(index, "indice_faiss.index")

with open("fragmentos.pkl", "wb") as f:
    pickle.dump(todos_los_fragmentos, f)

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

In [15]:
# Cargar modelo, índice y fragmentos
modelo = SentenceTransformer('all-MiniLM-L6-v2')
index = faiss.read_index("indice_faiss.index")
with open("fragmentos.pkl", "rb") as f:
    fragmentos = pickle.load(f)

def buscar_contexto(pregunta, top_k=3):
    embedding_pregunta = modelo.encode([pregunta])
    distancias, indices = index.search(np.array(embedding_pregunta), top_k)

    resultados = []
    for i in indices[0]:
        resultados.append(fragmentos[i])
    return resultados

In [16]:
resultados = buscar_contexto("¿Qué es un archivo CSV?")

for r in resultados:
    print(f"\n📄 {r['documento']}:\n{r['texto'][:500]}")


📄 ESP_Resumen_del_captulo_Vectorizacin_de_textos.pdf:
ub(r'[^a-zA-Z\']', ' ', text) 
" Me gustó este programa desde el primer episodio que vi que fue el  episodio  Rhapsody in 
Blue  para quienes no saben qué es  el Zan se vuelve loco y se convierte en pau episodio n
ivel    10   Los mejores efectos visuales y especiales que había visto en una serie de tel
evisión no hay nada parecido en ninguna parte  " 
Ahora no nos quedan más que las letras latinas y los espacios. En el siguiente paso, 
vamos a deshacernos de los espacios extra, ya que dificulta

📄 1_Resumen_del_captulo_Lectura_y_visualizacin_de_datos.pdf:
Resumen del capítulo: Lectura y visualización de datos
1
Resumen del capítulo: Lectura y 
visualización de datos
Solucionar problemas con archivos CSV
Recuerda que CSV significa valores separados por comas. Sin embargo, un archivo 
CSV no tiene que usar solo una coma como delimitador; se puede usar cualquier 
carácter. Por ejemplo, los valores separados por tabuladores son otro f